## Compound data types and External Resources in NWB

Some example code for referencing individual sub-elements of compound data types as fields for ERTs

In [1]:
import datetime
import inspect
import numpy as np
import pandas as pd
import os

from pynwb.spec import NWBNamespaceBuilder, export_spec, NWBGroupSpec, NWBAttributeSpec
from pynwb.spec import NWBDatasetSpec, NWBRefSpec, NWBDtypeSpec


from pynwb import NWBHDF5IO, NWBFile
from pynwb.core import DynamicTableRegion, DynamicTable
from pynwb.device import Device

from pynwb import register_class, load_namespaces
from hdmf.utils import docval, call_docval_func, getargs, get_docval


In [23]:
### oops
from hdmf.spec import DatasetSpec, DtypeSpec, NamespaceBuilder

parameter_dtype = [DtypeSpec(name='name',
                             dtype='text',
                             doc="Parameter name."),
                   DtypeSpec(name='value',
                             dtype='float',
                             doc='Parameter value.')
                  ]

parameter_spec = DatasetSpec(data_type_inc='Data',
                             data_type_def='Parameter',
                             doc='Parameter data type.',
                             quantity='?',
                             dtype=parameter_dtype
                            )

# create a builder for the namespace
ns_builder = NamespaceBuilder(name="ndx-parameter",doc="Extension for use in my laboratory",version='0.1')

# add the extension
ext_source = 'parameter.specs.yaml'
ns_builder.add_spec(ext_source, parameter_spec)

# save the namespace and extensions
ns_path = 'parameter.namespace.yaml'
ns_builder.export(ns_path)

In [7]:

parameters_spec = NWBDatasetSpec(neurodata_type_inc='NWBData',
                                 doc='Table for storing parameters.',
                                 quantity='?',
                                 dtype=parameter_dtype
                                )

parameter_table_spec = NWBGroupSpec(neurodata_type_inc='NWBData',
                                    neurodata_type_def='ParameterTable',
                                    doc='Table for storing ontologized parameters.',
                                    datasets=[parameters_spec]
                                    )

name = 'test_compound_data'
ns_path = name + ".namespace.yaml"
ext_source = name + ".extensions.yaml"

ns_builder = NWBNamespaceBuilder(name + ' extensions', name, version='0.1.0')
ns_builder.include_type('NWBContainer', namespace='core')
ns_builder.include_type('NWBData', namespace='core')
#ns_builder.include_type('NWBTable', namespace='core')
#ns_builder.include_type('VectorData', namespace='hdmf-common')
ns_builder.add_spec(ext_source, parameter_table_spec)
ns_builder.export(ns_path)

In [8]:
#from hdmf.common import get_class
#MyLocation = get_class(ns_path, 'test_compound_data')
#my_loc_inst = MyLocation()

from pynwb import get_class, load_namespaces

load_namespaces(ns_path)

param_ins = get_class('ParameterTable', 'test_compound_data')


ValueError: No specification for 'ElementIdentifiers' in namespace 'test_compound_data'

In [ ]:
param_ins.

The stimulus function table is designed to hold the names of stimulus waveforms and their relevant parameters - float parameters for numeric values and function parameters that reference other waveform functions (for parameter values that vary as a function of time).

In [ ]:
@register_class('Location')
class Location(NWBContainer):   
    
    __columns__ = ( 
                    {'name':'function_name', 
                     'description': 'The names of the 1D stimulus waveforms.',
                     'required': True,
                     'index': False},
                    {'name':'float_parameters', 
                     'description': 'The names of the float parameters for the 1D stimulus waveforms.',
                     'required': True,
                     'index': True},
                    {'name':'function_parameters',
                     'description': 'The function parameters for the 1D stimulus waveforms.',
                     'required': True,
                     'index': True}
    )
    
    
    @docval(*get_docval(DynamicTable.__init__, 'id', 'columns', 'colnames'))
    def __init__(self, **kwargs):
        kwargs['name'] = ('StimulusFunctionTable')
        kwargs['description'] = ('Table for storing ontologized 1D stimulus waveform metadata')
        call_docval_func(super().__init__, kwargs)
        

We create an example stimulus function table to hold a couple of square waves with associated parameters. We use the add_row function inherited from Dynamic Tables to add new entries.

In [ ]:
sft = StimulusFunctionTable()

float_params_sq1 = [('amplitude', -0.110, 'V'), 
                    ('duration', 0.500, 's'),
                    ('start_time', 0.1, 's')
                   ]

float_params_sq2 = [('amplitude', 0.090, 'V'), 
                    ('duration', 1, 's'),
                    ('start_time', 0.1, 's')
                   ]

float_params_ramp = [('amplitude', 0.200, 'V'), 
                     ('duration', 1, 's'),
                     ('start_time', 0.05, 's')
                    ]

float_params_sin = [('amplitude', 0.090, 'V'), 
                    ('duration', 1, 's'),
                    ('start_time', 0.1, 's')
                   ]

func_params_sin = [('frequency', 2, 'Hz')]


wave1 = {'function_name':'sq', 
         'float_parameters': float_params_sq1, 
         'function_parameters':[]}

wave2 = {'function_name':'sq', 
         'float_parameters': float_params_sq2, 
         'function_parameters':[]}

wave3 = {'function_name':'ramp', 
         'float_parameters': float_params_ramp, 
         'function_parameters':[]}

wave4 = {'function_name':'sin', 
         'float_parameters': float_params_sin, 
         'function_parameters': func_params_sin}


# Using add_row from DynamicTable

sft.add_row(data = wave1)
sft.add_row(data = wave2)
sft.add_row(data = wave3)
sft.add_row(data = wave4)


pd.set_option("display.max_colwidth", 100)

display(sft.to_dataframe())

## External Resources Table

- Resources: eg ontology we are referencing
- Entities: as defined by an entry in ontology/controlled terms
- Keys: reference from object into the ER tables that maps onto entities 
- Objects: thing we are ontologizing (eg SFT)


In [ ]:
from hdmf.common import ExternalResources
from hdmf import Container, Data
import pandas as pd

er = ExternalResources(name='ExtResTable')

The *StimulusFunctionTable* is the container object we will link to in the External Resources

In [ ]:
object_id = sft.object_id
print(object_id)

In [ ]:
# add_ref creates an entry across all tables (entities, keys, objects, resources)

er.add_ref(container = object_id, 
           field='', 
           key='sq',
           resource_name='Estim Ontology',
           resource_uri='Estim_Onto_uri',
           entity_id='Estim_square_ID',
           entity_uri='Estim_square_uri'
          )

er.add_ref(container = object_id, 
           field='', 
           key='duration',
           resource_name='Estim Ontology',
           resource_uri='Estim_Onto_uri',
           entity_id='Estim_duration_ID',
           entity_uri='Estim_duration_uri'
          )

er.add_ref(container = object_id, 
           field='', 
           key='amplitude',
           resource_name='Estim Ontology',
           resource_uri='Estim_Onto_uri',
           entity_id='Estim_amplitude_ID',
           entity_uri='Estim_amplitude_uri'
          )

er.add_ref(container = object_id, 
           field='', 
           key='start_time',
           resource_name='Estim Ontology',
           resource_uri='Estim_Onto_uri',
           entity_id='Estim_shift_ID',
           entity_uri='Estim_shift_uri'
          )


In [ ]:
er.resources.to_dataframe()

In [ ]:
er.keys.to_dataframe()

In [ ]:
er.entities.to_dataframe()

In [ ]:
k=er.add_key(key_name='sq')
er.keys.to_dataframe()

In [ ]:
key_object = er.get_key(key_name='sq')
print(key_object)

### Write

In [ ]:
from datetime import datetime
from dateutil.tz import tzlocal

start_time = datetime(2017, 4, 3, 11, tzinfo=tzlocal())
create_date = datetime(2017, 4, 15, 12, tzinfo=tzlocal())


nwbfile = NWBFile('demo', 'NWB456', start_time,
                  file_create_date=create_date)

nwbfile.add_acquisition(sft)

In [ ]:
# Write the SFT out to file

from pynwb import NWBHDF5IO

io = NWBHDF5IO('sft_ert.nwb', mode='w')
io.write(nwbfile)
io.close()

In [ ]:
# Reading in the file I just wrote

io = NWBHDF5IO('sft_ert.nwb', mode='r', load_namespaces=True)
sft_nwbfile = io.read()

In [ ]:
# print(sft_nwbfile.acquisition)

sft_in = sft_nwbfile.get_acquisition()

In [ ]:
sft_in.to_dataframe()

In [ ]:
sft_in['function_parameters'].target[0]

In [27]:
from pynwb import load_namespaces

namespace_path = 'parameter.namespace.yaml'
load_namespaces(namespace_path)

{'ndx-parameter': {}}

In [31]:
parameter_spec

hdmf.spec.spec.DatasetSpec